# The network energy intensity of video streaming over Wi-Fi and 4G

**Authors:** David Mytton, Iain Staffell, Malte Jansen.

**Institution:** Centre for Environmental Policy, Imperial College London, London, SW7 1NE, UK.

**Correspondence:** <david@davidmytton.co.uk>.

## Summary

> Between 2010-2018 internet traffic grew ten-fold and is expected to double again by 2022. With video streaming making up 60% of that traffic, 65% on mobile, it is important to understand its energy consumption. Here we develop an approach to assessing network energy consumption by collecting internet routing observations from volunteers, provide updated figures for the energy consumption of different sections of the internet, then calculate the network energy intensity of video streaming. We estimate the 2019 network energy footprint of the UK’s 39 billion hours of video streaming at 4.2 TWh, or 1.3% of total electricity generation. We show that the network energy intensity of video streaming over Wi-Fi is 0.091 kWh/hour, compared to 0.207 kWh/hour over 4G mobile internet.

## This notebook

This notebook walks through the methodology and calculations as described in the accompanying manuscript.

#### Imports

In [36]:
%pip install -r requirements.txt

import numpy as np
import pandas as pd
import pint
from pint import UnitRegistry

ureg = UnitRegistry()

## Internet video traffic

We assume YouTube is representative of all streaming because it is the most watched streaming service, and so has the most data available. Using YouTube as a case study, we can calculate the total amount of time, data volume, and therefore energy consumption of video streaming in general.

We therefore start with an assessment of YouTube.

### YouTube watch time

YouTube does not publish statistics so we start with watching figures from the advertising industry<sup>1</sup>, Ofcom<sup>2</sup>, and Netflix<sup>3</sup>.

In [2]:
# Dec 2019 UK YouTube minutes (mobile devices)
youtubeTimeDecDevices = 32099000000 * ureg.minutes # Source [1] (Pinpoint: pg 30)

# % watched YouTube (mobile devices)
youtubeWatchPercentageMobileDevices = 0.73 # 73% Source [2] (Pinpoint: Figure 4.10, pg116)

# % watched YouTube (mobile networks)
youtubeWatchPercentageMobileNetworks = 0.25 # 25% Source [3]

Using the total annual watch minutes, the total time watched on mobile devices and the total time watched on mobile networks, we can calculate the total time spent on Wi-Fi and non-mobile devices.

In [3]:
# Assume Dec 2019 figure is representative of the full year
youtubeTimeAnnualDevices = youtubeTimeDecDevices * 12

# Annual UK YouTube minutes (all networks, all devices)
youtubeTimeAnnualAllNetworksAllDevices = youtubeTimeAnnualDevices / youtubeWatchPercentageMobileDevices
print(f'Annual UK YouTube minutes (all networks, all devices): {youtubeTimeAnnualAllNetworksAllDevices:,.0f}')

print(f'Annual UK YouTube minutes (mobile devices): {youtubeTimeAnnualAllNetworksAllDevices:,.0f}')

# Annual UK YouTube minutes (mobile networks)
youtubeTimeAnnualMobileNetworks = youtubeWatchPercentageMobileNetworks * youtubeTimeAnnualAllNetworksAllDevices
print(f'Annual UK YouTube minutes (mobile networks): {youtubeTimeAnnualMobileNetworks:,.0f}')

# Annual UK YouTube minutes (Wi-fi networks, all devices)
youtubeTimeAnnualWifiAllDevices = youtubeTimeAnnualAllNetworksAllDevices - youtubeTimeAnnualMobileNetworks
print(f'Annual UK YouTube minutes (Wi-Fi networks, all devices): {youtubeTimeAnnualWifiAllDevices:,.0f}')

Annual UK YouTube minutes (all networks, all devices): 527,654,794,521 minute
Annual UK YouTube minutes (mobile devices): 527,654,794,521 minute
Annual UK YouTube minutes (mobile networks): 131,913,698,630 minute
Annual UK YouTube minutes (Wi-Fi networks, all devices): 395,741,095,890 minute


### YouTube data volume

The amount of data transmitted during a single YouTube video streaming session varies based on factors such as device type, screen size, video resolution, framerate, bit rate, network speed and which formats the video was encoded into. 

For each video quality setting, we take the mean average of the range of values reported in an observational assessment of 1 hour of streaming produced for an Android website<sup>4</sup>. In this assessment, data volume was recorded using three separate tools: Android's built-in data monitoring, Google's Datally app and the GlassWire data monitoring app.

In [4]:
dataVolume = pd.DataFrame({
    480: [0.48 * ureg.gigabyte, 0.66 * ureg.gigabyte], # 480p SD [min, max]
    720: [1.2 * ureg.gigabyte, 2.7 * ureg.gigabyte], # 720p HD [min ,max]
    1080: [2.5 * ureg.gigabyte, 4.1 * ureg.gigabyte], # 1080p FHD [min ,max]
    1440: [2.7 * ureg.gigabyte, 8.1 * ureg.gigabyte], # 1440p QHD [min ,max]
    2160: [5.5 * ureg.gigabyte, 23.0 * ureg.gigabyte], # 2160p UHD 4k [min ,max]
})

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1638: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


### Video traffic

We need to know how much traffic is from YouTube and video in general for both mobile and non-mobile.

In [5]:
internetTrafficYouTube = 0.118 # 11.8% Source [5] (Pinpoint: pg12)
internetTrafficVideo = 0.60 # 60% Source [5] (Pinpoint: pg6)
mobileTrafficYouTube = 0.271 # 27.1% Source [6] (Pinpoint: pg9)
mobileTrafficVideo = 0.655 # 65.5% Source [6] (Pinpoint: pg5)

## Video streaming watch time & data volume

Using the figures above, we calculate the total annual time spent and associated data volume for YouTube.

We assume mobile traffic is all 4G and non-mobile traffic is all Wi-Fi because 97% of UK premises have access to decent fixed (at least 10 Mbit/s) and 4G (at least 2 Mbit/s) services, 91% of geographic areas of the UK are able to receive 4G data service from at least one operator, and 4G carries 90% of UK mobile data traffic<sup>7</sup>. 3G connectivity is excluded because it carries only a small proportion of data traffic.

We assume the default video quality is 720p High Definition (HD) because during the 2020 Coronavirus pandemic, the default was reduced to 480p Standard Definition (SD)<sup>8</sup>. The reference year for this assessment is 2019.

In [6]:
# 720p HD video quality
videoQuality = 720

### YouTube

In [7]:
# Tidy up the calculations into an easily readable data frame
youtube = pd.DataFrame({
    'Time': {
        '4G': youtubeTimeAnnualMobileNetworks,
        'Wi-Fi': youtubeTimeAnnualWifiAllDevices,
        'Total': youtubeTimeAnnualMobileNetworks + youtubeTimeAnnualWifiAllDevices,
    },
    'Data': {
        '4G': youtubeTimeAnnualMobileNetworks * dataVolume[videoQuality].values.mean(),
        'Wi-Fi': youtubeTimeAnnualWifiAllDevices * dataVolume[videoQuality].values.mean(),
        'Total': (youtubeTimeAnnualMobileNetworks * dataVolume[videoQuality].values.mean()) + (youtubeTimeAnnualWifiAllDevices * dataVolume[videoQuality].values.mean()),
    }
})

# Format nicely for display
youtubeDisplay = pd.DataFrame({
    'Time': {
        '4G': f'{youtube["Time"]["4G"].to("hour"):,.0f} ({youtube["Time"]["4G"].to("hour").to_compact().magnitude:,.1f} bn)',
        'Wi-Fi': f'{youtube["Time"]["Wi-Fi"].to("hour"):,.0f} ({youtube["Time"]["Wi-Fi"].to("hour").to_compact().magnitude:,.1f} bn)',
        'Total': f'{youtube["Time"]["Total"].to("hour"):,.0f} ({youtube["Time"]["Total"].to("hour").to_compact().magnitude:,.1f} bn)',

    },
    'Data': {
        '4G': f'{youtube["Data"]["4G"].to("gigabyte hour"):,.0f~H} ({youtube["Data"]["4G"].to("exabyte hour"):,.1f~H})',
        'Wi-Fi': f'{youtube["Data"]["Wi-Fi"].to("gigabyte hour"):,.0f~H} ({youtube["Data"]["Wi-Fi"].to("exabyte hour"):,.1f~H})',
        'Total': f'{youtube["Data"]["Total"].to("gigabyte hour"):,.0f~H} ({youtube["Data"]["Total"].to("exabyte hour"):,.1f~H})',

    }
})
youtubeDisplay

,Time,Data
4G,"2,198,561,644 hour (2.2 bn)","4,287,195,205 GB hr (4.3 EB hr)"
Wi-Fi,"6,595,684,932 hour (6.6 bn)","12,861,585,616 GB hr (12.9 EB hr)"
Total,"8,794,246,575 hour (8.8 bn)","17,148,780,822 GB hr (17.1 EB hr)"


#### Validation by comparison

An Ofcom survey<sup>2</sup> reported the number of unique visitors to YouTube out of the UK population and how much time they spent on it per day in 2019. From this we can calculate the total viewing time and compare to our figure.

In [8]:
# YouTube daily visitors
ofcomYoutubeDailyVisitors = 41970000 # Source [2] (Pinpoint: Figure 4.6, pg111)

# YouTube average daily time spent
ofcomYoutubeAverageDailyTimeSpent = 35 * ureg.minutes # Source [2] (Pinpoint: Figure 4.8, pg 114)

# Annual UK YouTube time spent
ofcomYoutubeAnnualTimeSpent = ((ofcomYoutubeAverageDailyTimeSpent * 365) * ofcomYoutubeDailyVisitors)
print(f'Ofcom survey: {ofcomYoutubeAnnualTimeSpent.to("hours").magnitude:,.0f} ({ofcomYoutubeAnnualTimeSpent.to("hours").to_compact().magnitude:,.1f}bn) hours')
print(f'Our calculation: {youtube["Time"]["Total"].to("hours").magnitude:,.0f} ({youtube["Time"]["Total"].to("hours").to_compact().magnitude:,.1f}bn) hours')

# % difference
difference = ((youtube['Time']['Total'] - ofcomYoutubeAnnualTimeSpent) / youtube['Time']['Total']) * 100
print(f'Difference: {difference.magnitude:,.1f}%')

Ofcom survey: 8,936,112,500 (8.9bn) hours
Our calculation: 8,794,246,575 (8.8bn) hours
Difference: -1.6%


### All UK video streaming

Given that YouTube is a certain percentage of all internet traffic, and we know what percentage of traffic is video streaming, if we assume that YouTube is representative of all video streaming we can extrapolate for all UK video streaming.

In [9]:
# Extrapolate to all video streaming
allUKVideo4GData = (youtube['Data']['4G'] / mobileTrafficYouTube) * mobileTrafficVideo
allUKVideoWiFiData = (youtube['Data']['Wi-Fi'] / internetTrafficYouTube) * internetTrafficVideo
allUKVideo4GTime = allUKVideo4GData / dataVolume[videoQuality].values.mean()
allUKVideoWiFiTime = allUKVideoWiFiData / dataVolume[videoQuality].values.mean()

# Create a dataframe for easy analysis
allUKVideo = pd.DataFrame({
    'Time': {
        '4G': allUKVideo4GTime,
        'Wi-Fi': allUKVideoWiFiTime,
        'Total': allUKVideo4GTime + allUKVideoWiFiTime,
    },
    'Data': {
        '4G': allUKVideo4GData,
        'Wi-Fi': allUKVideoWiFiData,
        'Total': allUKVideo4GData + allUKVideoWiFiData,
    }
})

# Format nicely for display
allUKVideoDisplay = pd.DataFrame({
    'Time': {
        '4G': f'{allUKVideo["Time"]["4G"].to("hour"):,.0f} ({allUKVideo["Time"]["4G"].to("hour").to_compact().magnitude:,.1f} bn)',
        'Wi-Fi': f'{allUKVideo["Time"]["Wi-Fi"].to("hour"):,.0f} ({allUKVideo["Time"]["Wi-Fi"].to("hour").to_compact().magnitude:,.1f} bn)',
        'Total': f'{allUKVideo["Time"]["Total"].to("hour"):,.0f} ({allUKVideo["Time"]["Total"].to("hour").to_compact().magnitude:,.1f} bn)',

    },
    'Data': {
        '4G': f'{allUKVideo["Data"]["4G"].to("gigabyte hour"):,.0f~H} ({allUKVideo["Data"]["4G"].to("exabyte hour"):,.1f~H})',
        'Wi-Fi': f'{allUKVideo["Data"]["Wi-Fi"].to("gigabyte hour"):,.0f~H} ({allUKVideo["Data"]["Wi-Fi"].to("exabyte hour"):,.1f~H})',
        'Total': f'{allUKVideo["Data"]["Total"].to("gigabyte hour"):,.0f~H} ({allUKVideo["Data"]["Total"].to("exabyte hour"):,.1f~H})',

    }
})

allUKVideoDisplay

,Time,Data
4G,"5,313,866,704 hour (5.3 bn)","10,362,040,072 GB hr (10.4 EB hr)"
Wi-Fi,"33,537,381,008 hour (33.5 bn)","65,397,892,965 GB hr (65.4 EB hr)"
Total,"38,851,247,711 hour (38.9 bn)","75,759,933,037 GB hr (75.8 EB hr)"


#### Validation by comparison

An Ofcom survey<sup>9</sup> reported the total number of video minutes watched per day per person and the proportion of that which was not broadcast content i.e. online video streaming vs broadcast TV. Based on the UK population<sup>10</sup>, we can calculate the total viewing time and compare to our figure.

In [10]:
ofcomTotalVideoTimeDaily = 294  * ureg.minutes # Source [9] (Pinpoint: Figure 1.4, pg 16)
ofcomPercentageNonBroadcast = 0.31 # Source [9] (Pinpoint: Figure 1.4, pg 16)
ukPopulation = 66400000 # Source [10]

ofcomTotalVideoStreamingTime = (ofcomTotalVideoTimeDaily * ofcomPercentageNonBroadcast * ukPopulation * 365)

print(f'Ofcom survey: {ofcomTotalVideoStreamingTime.to("hours").magnitude:,.0f} ({ofcomTotalVideoStreamingTime.to("hours").to_compact().magnitude:,.1f}bn) hours')
print(f'Our calculation: {allUKVideo["Time"]["Total"].to("hours").magnitude:,.0f} ({allUKVideo["Time"]["Total"].to("hours").to_compact().magnitude:,.1f}bn) hours')

# % difference
difference = ((allUKVideo['Time']['Total'] - ofcomTotalVideoStreamingTime) / allUKVideo['Time']['Total']) * 100
print(f'Difference: {difference.magnitude:,.1f}%')

Ofcom survey: 36,814,484,000 (36.8bn) hours
Our calculation: 38,851,247,711 (38.9bn) hours
Difference: 5.2%


## Traceroute sample analysis

116 Scamper traceroute samples were returned by 29 participants. The individual samples are provided in `/traceroute-samples/samples/`. An aggregated CSV is provided in `/traceroute-samples/traceroute-samples.csv` with network ownership metadata returned by [IPInfo](https://ipinfo.io). Two samples returned anomalous results where zero hops were reported - these are excluded as `NaN`. See `/traceroute-samples/aggregate-samples.ipynb` for the aggregation code.

In [61]:
traceroutes = pd.read_csv('../traceroute-samples/traceroute-samples.csv')

# ASN Names
googleASN = 'AS15169 Google LLC'
facebookASN = 'AS32934 Facebook, Inc.'

# Filter by ASN
traceroutesISP = traceroutes.query(
    '`Destination ASN` != @googleASN and `Destination ASN` != @facebookASN')
traceroutesFBGOOG = traceroutes.query(
    '`Destination ASN` == @googleASN or `Destination ASN` == @facebookASN')

### Hop counts

Calculate the mean hope count and standard deviation based on the owner of the destination network. The ISP owning the destination network indicates a caching device is deployed within the network.

In [64]:
hopCounts = pd.DataFrame({
   'Count': {
        'All': traceroutes['Trace Hop Count'].count(),
        'ISP': traceroutesISP['Trace Hop Count'].count(),
        'FB & GOOG': traceroutesFBGOOG['Trace Hop Count'].count(),
    },
    'Mean': {
        'All': traceroutes['Trace Hop Count'].mean(skipna=True),
        'ISP': traceroutesISP['Trace Hop Count'].mean(skipna=True),
        'FB & GOOG': traceroutesFBGOOG['Trace Hop Count'].mean(skipna=True),
    },
    'StdDev': {
        'All': traceroutes['Trace Hop Count'].std(skipna=True),
        'ISP': traceroutesISP['Trace Hop Count'].std(skipna=True),
        'FB & GOOG': traceroutesFBGOOG['Trace Hop Count'].std(skipna=True),
    }
})

hopCounts

,Count,Mean,StdDev
All,114,7.412281,4.244333
ISP,52,6.096154,2.443556
FB & GOOG,62,8.516129,5.065932


## References

1. UKOM (2019) Q4 2019 UK Digital Market Overview report. Available from: https://ukom.uk.net/uploads/files/news/ukom/174/UKOM_Digital_Marketing_Overview_December_2019_final.pdf

2. Ofcom (2020) Online Nation – 2020 report. Available from: https://www.ofcom.org.uk/__data/assets/pdf_file/0027/196407/online-nation-2020-report.pdf

3. Solsman, J.E. (2018) Normally secretive Netflix inches back the curtain on how subscribers stream. 7 March 2018. CNET. Available from: https://www.cnet.com/news/netflix-shares-streaming-data-by-device-country-mobile-wi-fi-movies-tv/

4. Hindy, J. (2019) How much data does YouTube actually use? 30 June 2019. Android Authority. Available from: https://www.androidauthority.com/how-much-data-does-youtube-use-964560/

5. Sandvine (2020) The Mobile Internet Phenomena Report. Available from: https://www.sandvine.com/download-report-mobile-internet-phenomena-report-2020-sandvine

6. Sandvine (2019) The Global Internet Phenomena Report. Available from: https://www.sandvine.com/global-internet-phenomena-report-2019

7. Ofcom (2019) Connected Nations 2019. 20 December 2019. Ofcom. Available from: https://www.ofcom.org.uk/research-and-data/multi-sector-research/infrastructure-research/connected-nations-2019/main-report

8. Chee, F.Y. (2020) YouTube, Amazon Prime forgo streaming quality to relieve European networks. Reuters. 20 March. Available from: https://uk.reuters.com/article/us-health-coronavirus-youtube-exclusive-idUKKBN2170OP

9. Ofcom (2019) Media Nations: UK 2019. Available from: https://www.ofcom.org.uk/__data/assets/pdf_file/0019/160714/media-nations-2019-uk-report.pdf

10. Office for National Statistics (2019) Overview of the UK population. 23 August 2019. Available from: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/articles/overviewoftheukpopulation/august2019

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82ad941d-519a-4a94-b1a2-0a0958075b21' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>